In [19]:
import pandas as pd
import pickle
import os
from datetime import datetime

# Archivos
ARCHIVO_CSV = "cumples.csv"
ARCHIVO_PICKLE = "cumples.pkl"

# Función para cargar datos
def cargar_datos(modo="csv"):
    if modo == "csv" and os.path.exists(ARCHIVO_CSV):
        return pd.read_csv(ARCHIVO_CSV, parse_dates=["fecha"])
    elif modo == "pickle" and os.path.exists(ARCHIVO_PICKLE):
        with open(ARCHIVO_PICKLE, "rb") as f:
            return pickle.load(f)
    else:
        return pd.DataFrame(columns=["nombre", "fecha", "mensaje"])

# Función para guardar datos
def guardar_datos(df, modo="csv"):
    if modo == "csv":
        df.to_csv(ARCHIVO_CSV, index=False)
    elif modo == "pickle":
        with open(ARCHIVO_PICKLE, "wb") as f:
            pickle.dump(df, f)

# Días restantes
def dias_restantes(fecha):
    hoy = datetime.today()
    proximo = fecha.replace(year=hoy.year)
    if proximo < hoy:
        proximo = proximo.replace(year=hoy.year + 1)
    return (proximo - hoy).days

# Mostrar menú
def mostrar_menu():
    print("\n--- GESTOR DE CUMPLEAÑOS ---")
    print("1. Registrar cumpleaños")
    print("2. Ver próximos cumpleaños")
    print("3. Eliminar cumpleaños")
    print("4. Salir")

# Registrar nuevo cumpleaños
def registrar(df):
    nombre = input("Nombre: ").strip()
    fecha_str = input("Fecha de cumpleaños (YYYY-MM-DD): ").strip()
    mensaje = input("Mensaje personalizado (usa {nombre} como marcador): ").strip()
    try:
        fecha = pd.to_datetime(fecha_str)
        nuevo = pd.DataFrame([{
            "nombre": nombre,
            "fecha": fecha,
            "mensaje": mensaje
        }])
        df = pd.concat([df, nuevo], ignore_index=True)
        print(f"✅ Cumpleaños de {nombre} registrado.")
    except Exception as e:
        print("❌ Fecha inválida. Usa formato YYYY-MM-DD.")
    return df

# Mostrar cumpleaños
def mostrar(df):
    if df.empty:
        print("📭 No hay cumpleaños registrados.")
        return
    df["dias_restantes"] = df["fecha"].apply(dias_restantes)
    df_ordenado = df.sort_values("dias_restantes")
    for _, row in df_ordenado.iterrows():
        dias = row["dias_restantes"]
        nombre = row["nombre"]
        if dias == 0:
            mensaje = row["mensaje"].replace("{nombre}", nombre)
            print(f"🎉 ¡Hoy es el cumpleaños de {nombre}! Mensaje: {mensaje}")
        else:
            print(f"🎈 {nombre} cumple en {dias} días.")

# Eliminar cumpleaños
def eliminar(df):
    if df.empty:
        print("No hay cumpleaños para eliminar.")
        return df
    print("Cumpleaños registrados:")
    for i, row in df.iterrows():
        print(f"{i}. {row['nombre']} ({row['fecha'].date()})")
    try:
        indice = int(input("Número del cumpleaños a eliminar: "))
        if 0 <= indice < len(df):
            nombre = df.iloc[indice]["nombre"]
            df = df.drop(index=indice).reset_index(drop=True)
            print(f"🗑️ Cumpleaños de {nombre} eliminado.")
        else:
            print("❌ Índice inválido.")
    except ValueError:
        print("❌ Entrada inválida.")
    return df

# Programa principal
def main():
    modo = input("¿Modo de almacenamiento? (csv/pickle): ").strip().lower()
    if modo not in ["csv", "pickle"]:
        print("Modo no válido, usando CSV por defecto.")
        modo = "csv"
    df = cargar_datos(modo)

    while True:
        mostrar_menu()
        opcion = input("Elige una opción: ").strip()
        if opcion == "1":
            df = registrar(df)
            guardar_datos(df, modo)
        elif opcion == "2":
            mostrar(df)
        elif opcion == "3":
            df = eliminar(df)
            guardar_datos(df, modo)
        elif opcion == "4":
            print("👋 Saliendo del programa.")
            break
        else:
            print("❌ Opción inválida.")

if __name__ == "__main__":
    main()


¿Modo de almacenamiento? (csv/pickle): csv

--- GESTOR DE CUMPLEAÑOS ---
1. Registrar cumpleaños
2. Ver próximos cumpleaños
3. Eliminar cumpleaños
4. Salir
Elige una opción: 1
Nombre: Juan Perez
Fecha de cumpleaños (YYYY-MM-DD): 1999-12-12
Mensaje personalizado (usa {nombre} como marcador): {Juan} Muchas felicidades
✅ Cumpleaños de Juan Perez registrado.

--- GESTOR DE CUMPLEAÑOS ---
1. Registrar cumpleaños
2. Ver próximos cumpleaños
3. Eliminar cumpleaños
4. Salir
Elige una opción: 2
🎈 Juan Perez cumple en 211 días.
🎈 Juan Perez cumple en 211 días.

--- GESTOR DE CUMPLEAÑOS ---
1. Registrar cumpleaños
2. Ver próximos cumpleaños
3. Eliminar cumpleaños
4. Salir


KeyboardInterrupt: Interrupted by user